<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/%E2%8A%BF%E2%8A%BFCT_qPCR_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# ---- 入力データ ----
data = {
    'Sample': ['Control1', 'Control2','Control3',
               'Treated1', 'Treated2', 'Treated3',
               'Treated_1_1', 'Treated1_2','Treated_1_3',
               'Treated2_1', 'Treated2_2', 'Treated2_3'],
    'Target_Ct': [40.26, 44.19, 35.94, 0, 35.65, 35.18, 35.19, 35.23, 0, 36.78, 38.25, 0],
    'Housekeeping1_Ct': [25.8, 30.03, 21.82, 32.07, 19.39, 19.6, 19.85, 18.26, 30.45, 19.49, 23.08, 30.99],
    'Housekeeping2_Ct': [25.8, 30.03, 21.82, 32.07, 19.39, 19.6, 19.85, 18.26, 30.45, 19.49, 23.08, 30.99]
}
df = pd.DataFrame(data)

# ---- 条件ラベル ----
df['Condition'] = np.where(df['Sample'].str.contains('Control', case=False), 'Control', 'Treated')

# ---- 未検出(0)を NaN に置換 ----
for col in ['Target_Ct', 'Housekeeping1_Ct', 'Housekeeping2_Ct']:
    df[col] = df[col].replace(0, np.nan)

# ---- ΔCt ----
df['DeltaCt_HK1'] = df['Target_Ct'] - df['Housekeeping1_Ct']
df['DeltaCt_HK2'] = df['Target_Ct'] - df['Housekeeping2_Ct']

# ---- Control群の平均ΔCt ----
ctrl_mask = (df['Condition'] == 'Control')
ctrl_mean_dCt_HK1 = df.loc[ctrl_mask, 'DeltaCt_HK1'].mean(skipna=True)
ctrl_mean_dCt_HK2 = df.loc[ctrl_mask, 'DeltaCt_HK2'].mean(skipna=True)

# ---- ΔΔCt と相対発現 ----
df['DeltaDeltaCt_HK1'] = df['DeltaCt_HK1'] - ctrl_mean_dCt_HK1
df['DeltaDeltaCt_HK2'] = df['DeltaCt_HK2'] - ctrl_mean_dCt_HK2
df['RelExp_HK1'] = 2 ** (-df['DeltaDeltaCt_HK1'])
df['RelExp_HK2'] = 2 ** (-df['DeltaDeltaCt_HK2'])

# ---- 幾何平均（2つのHKを統合した発現量）----
def geo_mean(values):
    values = np.array(values, dtype=float)
    values = values[~np.isnan(values)]
    return np.exp(np.mean(np.log(values))) if len(values) else np.nan

df['RelExp_GeoMean'] = [
    geo_mean([r1, r2]) for r1, r2 in zip(df['RelExp_HK1'], df['RelExp_HK2'])
]

# ---- 未検出の行を除外 ----
out = df[df['Target_Ct'].notna()].copy()

# ---- 出力 ----
cols_show = ['Sample','Condition','Target_Ct','Housekeeping1_Ct','Housekeeping2_Ct',
             'DeltaCt_HK1','DeltaCt_HK2','DeltaDeltaCt_HK1','DeltaDeltaCt_HK2',
             'RelExp_HK1','RelExp_HK2','RelExp_GeoMean']
print(out[cols_show])

# ---- 要約（Controlを1としたときのFold Change）----
summary = out.groupby('Condition')['RelExp_GeoMean'].agg(lambda s: geo_mean(s)).rename('GeoMean_FC')
# 正規化：Controlを1にスケーリング
summary = summary / summary.loc['Control']
print("\nCondition-wise relative expression (Control = 1):")
print(summary)


         Sample Condition  Target_Ct  Housekeeping1_Ct  Housekeeping2_Ct  \
0      Control1   Control      40.26             25.80             25.80   
1      Control2   Control      44.19             30.03             30.03   
2      Control3   Control      35.94             21.82             21.82   
4      Treated2   Treated      35.65             19.39             19.39   
5      Treated3   Treated      35.18             19.60             19.60   
6   Treated_1_1   Treated      35.19             19.85             19.85   
7    Treated1_2   Treated      35.23             18.26             18.26   
9    Treated2_1   Treated      36.78             19.49             19.49   
10   Treated2_2   Treated      38.25             23.08             23.08   

    DeltaCt_HK1  DeltaCt_HK2  DeltaDeltaCt_HK1  DeltaDeltaCt_HK2  RelExp_HK1  \
0         14.46        14.46          0.213333          0.213333    0.862542   
1         14.16        14.16         -0.086667         -0.086667    1.061914   